In [1]:
from collections import defaultdict
import time
import random
import dynet as dy
import numpy as np

# Functions to read in the corpus
w2i = defaultdict(lambda: len(w2i))
t2i = defaultdict(lambda: len(t2i))
UNK = w2i["<unk>"]
def read_dataset(filename):
    with open(filename, "r") as f:
        for line in f:
            tag, words = line.lower().strip().split(" ||| ")
            yield ([w2i[x] for x in words.split(" ")], t2i[tag])

# Read in the data
train = list(read_dataset("../data/classes/train.txt"))
w2i = defaultdict(lambda: UNK, w2i)
dev = list(read_dataset("../data/classes/test.txt"))
nwords = len(w2i)
ntags = len(t2i)

# Start DyNet and define trainer
model = dy.Model()
trainer = dy.AdamTrainer(model)

In [2]:
# Define the model
EMB_SIZE = 64
W_emb = model.add_lookup_parameters((nwords, EMB_SIZE)) # Word embeddings
W_sm = model.add_parameters((ntags, EMB_SIZE))          # Softmax weights
b_sm = model.add_parameters((ntags))                      # Softmax bias

In [3]:
# A function to calculate scores for one value
def calc_scores(words):
    dy.renew_cg()
    cbow = dy.esum([dy.lookup(W_emb, x) for x in words])
    W_sm_exp = dy.parameter(W_sm)
    b_sm_exp = dy.parameter(b_sm)
    return W_sm_exp * cbow + b_sm_exp

In [4]:
for ITER in range(100):
    # Perform training
    random.shuffle(train)
    train_loss = 0.0
    start = time.time()
    for words, tag in train:
        my_loss = dy.pickneglogsoftmax(calc_scores(words), tag)
        train_loss += my_loss.value()
        my_loss.backward()
    trainer.update()
    print("iter %r: train loss/sent=%.4f, time=%.2fs" % (ITER, train_loss/len(train), time.time()-start))
    # Perform testing
    test_correct = 0.0
    for words, tag in dev:
        scores = calc_scores(words).npvalue()
        predict = np.argmax(scores)
        if predict == tag:
              test_correct += 1
    print("iter %r: test acc=%.4f" % (ITER, test_correct/len(dev)))

iter 0: train loss/sent=1.8580, time=0.40s
iter 0: test acc=0.2149
iter 1: train loss/sent=1.7948, time=0.38s
iter 1: test acc=0.2308
iter 2: train loss/sent=1.7384, time=0.38s
iter 2: test acc=0.2321
iter 3: train loss/sent=1.6884, time=0.38s
iter 3: test acc=0.2439
iter 4: train loss/sent=1.6441, time=0.38s
iter 4: test acc=0.2430
iter 5: train loss/sent=1.6047, time=0.39s
iter 5: test acc=0.2439
iter 6: train loss/sent=1.5693, time=0.53s
iter 6: test acc=0.2507
iter 7: train loss/sent=1.5365, time=0.39s
iter 7: test acc=0.2570
iter 8: train loss/sent=1.5048, time=0.39s
iter 8: test acc=0.2638
iter 9: train loss/sent=1.4736, time=0.40s
iter 9: test acc=0.2670
iter 10: train loss/sent=1.4424, time=0.42s
iter 10: test acc=0.2724
iter 11: train loss/sent=1.4114, time=0.69s
iter 11: test acc=0.2765
iter 12: train loss/sent=1.3806, time=0.64s
iter 12: test acc=0.2833
iter 13: train loss/sent=1.3503, time=0.38s
iter 13: test acc=0.2860
iter 14: train loss/sent=1.3204, time=0.37s
iter 14: t